### Dependecias 

In [142]:
import os
import re
from azure.ai.documentintelligence import DocumentIntelligenceClient
from azure.ai.documentintelligence.models import AnalyzeDocumentRequest
from azure.core.credentials import AzureKeyCredential
from azure.storage.blob import BlobServiceClient
import json

### Funcion de analisis de texto

In [145]:
def analyze_read(endpoint, key, name):

    formUrl = f"https://fs19920811dev.blob.core.windows.net/bronze/{name}"

    try:
        """Extrae texto de un PDF utilizando Azure Document Intelligence."""
        document_intelligence_client = DocumentIntelligenceClient(
            endpoint=endpoint, credential=AzureKeyCredential(key)
        )
        poller = document_intelligence_client.begin_analyze_document(
            "prebuilt-read", AnalyzeDocumentRequest(url_source=formUrl)
        )
        result = poller.result()
        print(f"Archivo {name} analizado correctamente.")

        return result.content
    
    except Exception as e:
        print(f"Error al analizar el archivo {name}: {e}")

### Funcion para normalizar texto

In [147]:
def normalize_text(original_text):
    """
    Normaliza el texto original para prepararlo para chunking jerárquico,
    sin alterar la estructura de encabezados ni cortar links.
    """
    lines = original_text.splitlines()
    normalized_lines = []

    # Expresiones regulares que detectan encabezados
    re_letter_header = re.compile(r'^[A-Z]\.\s+.*')
    re_number_header = re.compile(r'^\d+\.\s+.*')

    i = 0
    while i < len(lines):
        line = lines[i].rstrip("\r\n")  # elimina CRLF y whitespace final
        # Limpieza básica de secuencias de espacios
        line = re.sub(r'\s{2,}', ' ', line)

        # Si la línea queda vacía, omitir (opcional)
        if not line.strip():
            i += 1
            continue

        # 1. Detectar links cortados (URL en dos líneas) [opcional si lo necesitas]
        #    Si una línea contiene 'https://' y la siguiente línea no es encabezado,
        #    quizás quieras concatenarla.
        #    (Ejemplo práctico. Ajusta lógica según tu caso real.)
        if "https://" in line and not re_letter_header.match(line) and not re_number_header.match(line):
            # Revisar si la URL está cortada. Por ejemplo, si no empieza con https sino
            # que es "🔗 https://maps.app.goo.gl/" en esta línea y "RestoURL" en la siguiente
            # Normalmente, si la siguiente línea no es encabezado, podríamos unirla.
            # La heurística aquí es flexible y depende de tu dataset.
            pass  # Para no complicar, no haremos nada aquí. Lo dejamos de ejemplo.

        # 2. Agregar la línea normalizada
        normalized_lines.append(line.strip())
        i += 1

    # Reconstruir texto
    cleaned_text = "\n".join(normalized_lines)
    return cleaned_text


### Funcion de segmentacion

In [149]:
def segment_text_as_json(text):
    
    """
        Crea una estructura jerárquica:
          {
            "A. ...": {
              "1. ...": [...],
              "2. ...": [...]
            },
            "B. ...": {
              "1. ...": [...],
              ...
            },
            ...
          }
        """
    
    def create_hierarchical_chunking(text):
        
        # Patrón para detectar encabezados de tipo 'X. Algo...' donde X puede ser A, B, C ... o 1, 2, 3 ...
        # 1) Letra mayúscula seguida de punto: ^[A-Z]\.\s+(.*)
        # 2) Número (uno o más dígitos) seguido de punto: ^\d+\.\s+(.*)
        letter_pattern = re.compile(r'^([A-Z]\.\s+.*)')
        number_pattern = re.compile(r'^(\d+\.\s+.*)')
    
        chunk_dict = {}
        current_letter = None
        current_number = None
    
        # Procesamos línea por línea
        for line in text.splitlines():
            line = line.strip()
            if not line:
                # Si la línea está en blanco, se ignora (no aporta contenido)
                continue
    
            # Coincide con encabezado de Letra (e.g. "A. Texto", "B. Texto", etc.)
            letter_match = letter_pattern.match(line)
            # Coincide con encabezado de Número (e.g. "1. Texto", "2. Texto", etc.)
            number_match = number_pattern.match(line)
    
            if letter_match:
                # Nuevo gran bloque (ej. "A. Nombres de los novios.")
                current_letter = letter_match.group(1)
                # Creamos un diccionario vacío para esa letra
                chunk_dict[current_letter] = {}
                # Reseteamos el sub-bloque
                current_number = None
    
            elif number_match:
                # Sub-bloque dentro de la letra (ej. "1. Novia: Laura...")
                if current_letter is not None:
                    current_number = number_match.group(1)
                    # Creamos una lista para almacenar líneas que pertenezcan a este sub-bloque
                    chunk_dict[current_letter][current_number] = []
                else:
                    # Si no hay letra definida, se ignora o maneja según tu lógica
                    pass
            else:
                # No es encabezado de letra ni de número: se agrega como línea de contenido
                # al último sub-bloque (letter->number) detectado
                if current_letter is not None and current_number is not None:
                    chunk_dict[current_letter][current_number].append(line)
                else:
                    # Si llegamos aquí sin 'current_letter' o sin 'current_number',
                    # significa que la línea no está bajo ningún encabezado válido.
                    # Se puede ignorar o manejar de otra forma.
                    pass
    
        return chunk_dict
    
    # 2) Creamos la estructura de chunking
    hierarchical_chunking = create_hierarchical_chunking(text)
        
    # 3) Convertimos a JSON para observar el resultado final
    # y lo imprimimos (o lo guardamos, según tu caso de uso)
        
    json_result = json.dumps(
        hierarchical_chunking,
        ensure_ascii=False,
        indent=2)
    
    return json_result

### Funcion para descargar archivos desde un blob

In [151]:
def download_from_blob(connection_string, container_name, blob_name):

    try:
        """Descarga los datos desde un Blob Storage via Connection String"""
        blob_service_client = BlobServiceClient.from_connection_string(connection_string)
        blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
        blob_text = blob_client.download_blob().content_as_text()
        print(f"Archivo descargado exitosamente")
        return blob_text
    except Exception as e:
        print(f"Error al descargar el archivo: {e}")

### Funcion para subir archivos a un blob

In [153]:
def upload_to_blob(connection_string, container_name, blob_name, data):

    try:
        """Sube datos a un Blob Storage de Azure utilizando Connection String."""
        blob_service_client = BlobServiceClient.from_connection_string(connection_string)
        blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
        blob_client.upload_blob(data, overwrite=True)
        print(f"Archivo subido exitosamente a {blob_name}")
    except Exception as e:
        print(f"Error al subir el archivo {blob_name}: {e}")

### Funcion para convertir json de texto a diccionario

In [155]:
def string_to_json(json_string):
    try:
        json_data = json.loads(json_string)  # Convertir string a JSON
        return json_data
        
    except json.JSONDecodeError as e:
        print(f"Error al convertir string a JSON: {e}")
        return None

### PDF a texto

In [157]:
# Credentials
endpoint = "https://di19920811dev.cognitiveservices.azure.com/"
key = "6QC6o06093lvB0vAbbcJ9F8ay58jTj9NMPc60kM7dJhIiHpP3TJIJQQJ99AKACYeBjFXJ3w3AAALACOGEMje"

# Values
name = "instrucciones.pdf"
connection_string = "DefaultEndpointsProtocol=https;AccountName=fs19920811dev;AccountKey=+EA7jcWlksfpZnod9P+5+9yRlhMf2EFa7ulsO04+Je7w3UyXD/pTL2jCnOGjjJ4THXIp6qEvmDP4+AStVVtjSg==;EndpointSuffix=core.windows.net"
output_container = "silver"
file_name = os.path.splitext(name)[0]
file_name = f"{file_name}.txt"

# Procesa el PDF y guarda el resultado en Blob
pdf_text = analyze_read(endpoint, key, name)
upload_to_blob(connection_string, output_container, file_name, pdf_text)
print(f"El archivo {file_name} ha sido cargado exitosamente.")

Archivo instrucciones.pdf analizado correctamente.
Archivo subido exitosamente a instrucciones.txt
El archivo instrucciones.txt ha sido cargado exitosamente.


In [158]:
print(pdf_text)

A. Nombres de los novios.
1. Novia: Laura Guadalupe Zarazúa Arvizu.
2. Novio: José Alberto Lozano Sánchez.
B. Cuál es la temática de la bosa?
1. Temática de la boda
- Viajes y ciudades del mundo
C. Actividades de la boda / fiesta.
1. Lista de actividades
- Banquete
- Proyección de video de los novios
- Música en vivo
- Rifa de centros de mesa
- Lanzamiento de ramo
- DJ
- Etc.
D. Código de vestimenta (Etiqueta).
1. Vestimenta y recomendaciones adicionales
- Hombres: Traje o guayabera elegante.
- Mujeres: Vestido formal o de cóctel.
- Evitar colores blancos o beige.
- Procura llevar *zapatos cerrados o de piso* para la recepción.
- Lleva un *suéter o ropa abrigadora* para la noche.
- * No molestar ni aventar comida a los perritos* que estarán en su corral.
- Si necesitas maquillarte, consulta con tiempo a la novia para hacer una *reservación en salones de belleza locales *.
- ¡ Si te animas, puedes traer tu *casa de campaña *! Hay espacio en el terreno para acampar y continuar la fiesta.

### Normalizar texto

In [160]:
try:
    connection_string = "DefaultEndpointsProtocol=https;AccountName=fs19920811dev;AccountKey=+EA7jcWlksfpZnod9P+5+9yRlhMf2EFa7ulsO04+Je7w3UyXD/pTL2jCnOGjjJ4THXIp6qEvmDP4+AStVVtjSg==;EndpointSuffix=core.windows.net"
    container_name = "silver"
    input_blob_name = "instrucciones.txt"
    output_blob_name = "instrucciones_cleaned.txt"

    # Obtener el contenido del blob original
    blob_text = download_from_blob(connection_string, container_name, input_blob_name)
    # Limpiar el texto
    cleaned_text = normalize_text(blob_text)
    # Guardar el texto limpio en un nuevo archivo en Blob Storage
    upload_to_blob(connection_string, container_name, output_blob_name, cleaned_text)

    print(f"El archivo {output_blob_name} ha sido generado exitosamente en silver.")

except Exception as e:
    print(f"Error al limpiar y guardar el archivo {output_blob_name}: {e}")       

Archivo descargado exitosamente
Archivo subido exitosamente a instrucciones_cleaned.txt
El archivo instrucciones_cleaned.txt ha sido generado exitosamente en silver.


In [161]:
print(cleaned_text)

A. Nombres de los novios.
1. Novia: Laura Guadalupe Zarazúa Arvizu.
2. Novio: José Alberto Lozano Sánchez.
B. Cuál es la temática de la bosa?
1. Temática de la boda
- Viajes y ciudades del mundo
C. Actividades de la boda / fiesta.
1. Lista de actividades
- Banquete
- Proyección de video de los novios
- Música en vivo
- Rifa de centros de mesa
- Lanzamiento de ramo
- DJ
- Etc.
D. Código de vestimenta (Etiqueta).
1. Vestimenta y recomendaciones adicionales
- Hombres: Traje o guayabera elegante.
- Mujeres: Vestido formal o de cóctel.
- Evitar colores blancos o beige.
- Procura llevar *zapatos cerrados o de piso* para la recepción.
- Lleva un *suéter o ropa abrigadora* para la noche.
- * No molestar ni aventar comida a los perritos* que estarán en su corral.
- Si necesitas maquillarte, consulta con tiempo a la novia para hacer una *reservación en salones de belleza locales *.
- ¡ Si te animas, puedes traer tu *casa de campaña *! Hay espacio en el terreno para acampar y continuar la fiesta.

### Chunking

In [163]:
try:
    connection_string = "DefaultEndpointsProtocol=https;AccountName=fs19920811dev;AccountKey=+EA7jcWlksfpZnod9P+5+9yRlhMf2EFa7ulsO04+Je7w3UyXD/pTL2jCnOGjjJ4THXIp6qEvmDP4+AStVVtjSg==;EndpointSuffix=core.windows.net"
    input_container_name = "silver"
    output_container_name = "gold"
    input_blob_name = "instrucciones_cleaned.txt" 
    output_blob_name = "instrucciones_segmented.json"
    
    # Obtener el contenido del blob original
    blob_text = download_from_blob(connection_string, input_container_name, input_blob_name)
    # Segmentar el texto
    chunks = segment_text_as_json(blob_text)
    # Guardar el texto limpio en un nuevo archivo en Blob Storage
    upload_to_blob(connection_string, output_container_name, output_blob_name, chunks)
    print(f"El archivo {output_blob_name} ha sido generado exitosamente en gold.")

except Exception as e:
    print(f"Error en la segmentación del archivo: {e}")

Archivo descargado exitosamente
Archivo subido exitosamente a instrucciones_segmented.json
El archivo instrucciones_segmented.json ha sido generado exitosamente en gold.


In [164]:
print(chunks)

{
  "A. Nombres de los novios.": {
    "1. Novia: Laura Guadalupe Zarazúa Arvizu.": [],
    "2. Novio: José Alberto Lozano Sánchez.": []
  },
  "B. Cuál es la temática de la bosa?": {
    "1. Temática de la boda": [
      "- Viajes y ciudades del mundo"
    ]
  },
  "C. Actividades de la boda / fiesta.": {
    "1. Lista de actividades": [
      "- Banquete",
      "- Proyección de video de los novios",
      "- Música en vivo",
      "- Rifa de centros de mesa",
      "- Lanzamiento de ramo",
      "- DJ",
      "- Etc."
    ]
  },
  "D. Código de vestimenta (Etiqueta).": {
    "1. Vestimenta y recomendaciones adicionales": [
      "- Hombres: Traje o guayabera elegante.",
      "- Mujeres: Vestido formal o de cóctel.",
      "- Evitar colores blancos o beige.",
      "- Procura llevar *zapatos cerrados o de piso* para la recepción.",
      "- Lleva un *suéter o ropa abrigadora* para la noche.",
      "- * No molestar ni aventar comida a los perritos* que estarán en su corral.",
      

### Upload Chunks json into AI Search index 

In [166]:
import json
import requests
import openai
import os
import uuid
from openai import AzureOpenAI

In [167]:
embedding_model = "text-embedding-ada-002"

#creating an Azure OpenAI client
client = AzureOpenAI(
  api_key = "0ee0741378b24238b0db0d9c2b7c6b2c",  
  api_version = "2024-02-15-preview",
  azure_endpoint = "https://ai19920811dv0.openai.azure.com/" 
)

In [168]:
# Configuración de Azure Search
search_service_endpoint = "https://aisearch19920811dev.search.windows.net"
index_name = "wedding-info-index"
search_api_key = "UscTnxkGQQe4aJx3cRwMGlWWgj1ElG5KjKfcnyzEvAAzSeCLa0SG"
api_search_version = "2024-11-01-preview"

headers = {
    "Content-Type": "application/json",
    "api-key": search_api_key
}

In [169]:
# Embedding function
def generate_embedding(client, text, embedding_model):
    
    response = client.embeddings.create(
        input=text,
        model = embedding_model
    )
    
    embeddings=response.model_dump()
    return embeddings['data'][0]['embedding']

In [170]:
# Descargar jsons desde GOLD
connection_string = "DefaultEndpointsProtocol=https;AccountName=fs19920811dev;AccountKey=+EA7jcWlksfpZnod9P+5+9yRlhMf2EFa7ulsO04+Je7w3UyXD/pTL2jCnOGjjJ4THXIp6qEvmDP4+AStVVtjSg==;EndpointSuffix=core.windows.net"
container_name = "gold"
chunks_blob = "instrucciones_segmented.json" 
index_def_blob = "index_definition.json" 

try:
    chunks = download_from_blob(connection_string, container_name, chunks_blob)
    print(f"El archivo {chunks_blob} ha sido leido exitosamente desde gold.")

except Exception as e:
    print(f"Error al leer archivo {chunks_blob}: {e}")

try:
    index_definition = download_from_blob(connection_string, container_name, index_def_blob)
    print(f"El archivo {index_def_blob} ha sido leido exitosamente desde gold.")

except Exception as e:
    print(f"Error al leer archivo {index_def_blob}: {e}")

Archivo descargado exitosamente
El archivo instrucciones_segmented.json ha sido leido exitosamente desde gold.
Archivo descargado exitosamente
El archivo index_definition.json ha sido leido exitosamente desde gold.


In [171]:
def get_index_document(client, chunks, embedding_model):
            
    data = string_to_json(chunks)
    documents = []
    for section_title, subsections in data.items():
        for subsection_title, content_list in subsections.items():
            content_text = " ".join(content_list) if isinstance(content_list, list) else content_list
            text_4_vector = section_title + " " + subsection_title 
            vector = generate_embedding(client, text_4_vector, embedding_model)

            doc = {
                "id": str(uuid.uuid4()),
                "title": section_title,
                "subtitle": subsection_title,
                "content": content_text,
                "contentVector": vector,
                "category": "Instrucciones" if "Instrucciones" in section_title else "General",
                "additionalMetadata": "origen=instrucciones_segmented.json"
            }
            documents.append(doc)
            
    return documents

In [172]:
documents = get_index_document(client, chunks, embedding_model)

In [173]:
len(documents)

48

### Cuerpo de la definicion del Index

```json
{
  "@odata.etag": "\"0x8DD5E1545012FF4\"",
  "name": "wedding-info-index",
  "fields": [
    {
      "name": "id",
      "type": "Edm.String",
      "searchable": false,
      "filterable": false,
      "retrievable": true,
      "stored": true,
      "sortable": false,
      "facetable": false,
      "key": true,
      "synonymMaps": []
    },
    {
      "name": "title",
      "type": "Edm.String",
      "searchable": true,
      "filterable": true,
      "retrievable": true,
      "stored": true,
      "sortable": false,
      "facetable": false,
      "key": false,
      "analyzer": "standard.lucene",
      "synonymMaps": []
    },
    {
      "name": "subtitle",
      "type": "Edm.String",
      "searchable": true,
      "filterable": true,
      "retrievable": true,
      "stored": true,
      "sortable": false,
      "facetable": false,
      "key": false,
      "analyzer": "standard.lucene",
      "synonymMaps": []
    },
    {
      "name": "content",
      "type": "Edm.String",
      "searchable": true,
      "filterable": false,
      "retrievable": true,
      "stored": true,
      "sortable": false,
      "facetable": false,
      "key": false,
      "analyzer": "standard.lucene",
      "synonymMaps": []
    },
    {
      "name": "contentVector",
      "type": "Collection(Edm.Single)",
      "searchable": true,
      "filterable": false,
      "retrievable": false,
      "stored": true,
      "sortable": false,
      "facetable": false,
      "key": false,
      "dimensions": 1536,
      "vectorSearchProfile": "vector-profile-19920811",
      "synonymMaps": []
    },
    {
      "name": "category",
      "type": "Edm.String",
      "searchable": true,
      "filterable": true,
      "retrievable": false,
      "stored": true,
      "sortable": false,
      "facetable": true,
      "key": false,
      "analyzer": "standard.lucene",
      "synonymMaps": []
    },
    {
      "name": "additionalMetadata",
      "type": "Edm.String",
      "searchable": false,
      "filterable": true,
      "retrievable": false,
      "stored": true,
      "sortable": false,
      "facetable": false,
      "key": false,
      "synonymMaps": []
    }
  ],
  "scoringProfiles": [],
  "suggesters": [],
  "analyzers": [],
  "normalizers": [],
  "tokenizers": [],
  "tokenFilters": [],
  "charFilters": [],
  "similarity": {
    "@odata.type": "#Microsoft.Azure.Search.BM25Similarity"
  },
  "vectorSearch": {
    "algorithms": [
      {
        "name": "vector-config-19920811",
        "kind": "hnsw",
        "hnswParameters": {
          "metric": "cosine",
          "m": 4,
          "efConstruction": 400,
          "efSearch": 500
        }
      }
    ],
    "profiles": [
      {
        "name": "vector-profile-19920811",
        "algorithm": "vector-config-19920811"
      }
    ],
    "vectorizers": [],
    "compressions": []
  }
}

### Eliminar el indice previo

In [176]:
# URL del el índice
url = f"{search_service_endpoint}/indexes/{index_name}?api-version={api_search_version}"

# Enviar la solicitud `DELETE` para eliminar el indice previo
response = requests.delete(url, headers=headers)

# Verificar si la solicitud fue exitosa
if response.status_code == 204:
    print(f"Índice '{index_name}' eliminado correctamente en Azure AI Search.")
elif response.status_code == 404:
    print(f"Índice '{index_name}' no encontrado. Puede que ya haya sido eliminado.")
else:
    print(f"Error al eliminar el índice: {response.text}")

Índice 'wedding-info-index' eliminado correctamente en Azure AI Search.


### Crear la definicion del nuevo indice

In [178]:
# Definicion del index
index_def_json = string_to_json(index_definition)
# Enviar la solicitud `PUT` para crear el índice
response = requests.put(url, headers=headers, json=index_def_json)

# Verificar si la solicitud fue exitosa
if response.status_code == 201:
    print(f"Índice '{index_name}' creado correctamente en Azure AI Search.")
elif response.status_code == 204:
    print(f"Índice '{index_name}' ya existía y se actualizó correctamente.")
else:
    print(f"Error al crear el índice: {response.text}")

Índice 'wedding-info-index' creado correctamente en Azure AI Search.


### Subir documentos al indice

In [180]:
# Subir datos al index por lotes
batch_size = 1
for i in range(0, len(documents), batch_size):
    batch = {"value": documents[i:i+batch_size]}
    url = f"{search_service_endpoint}/indexes/{index_name}/docs/index?api-version={api_search_version}"
    response = requests.post(url, headers=headers, json=batch)
    if response.status_code == 200:
        print(f"Batch {i}-{i+len(batch['value'])} subido correctamente.")
    else:
        print(f"Error en batch {i}: {response.status_code}")

Batch 0-1 subido correctamente.
Batch 1-2 subido correctamente.
Batch 2-3 subido correctamente.
Batch 3-4 subido correctamente.
Batch 4-5 subido correctamente.
Batch 5-6 subido correctamente.
Batch 6-7 subido correctamente.
Batch 7-8 subido correctamente.
Batch 8-9 subido correctamente.
Batch 9-10 subido correctamente.
Batch 10-11 subido correctamente.
Batch 11-12 subido correctamente.
Batch 12-13 subido correctamente.
Batch 13-14 subido correctamente.
Batch 14-15 subido correctamente.
Batch 15-16 subido correctamente.
Batch 16-17 subido correctamente.
Batch 17-18 subido correctamente.
Batch 18-19 subido correctamente.
Batch 19-20 subido correctamente.
Batch 20-21 subido correctamente.
Batch 21-22 subido correctamente.
Batch 22-23 subido correctamente.
Batch 23-24 subido correctamente.
Batch 24-25 subido correctamente.
Batch 25-26 subido correctamente.
Batch 26-27 subido correctamente.
Batch 27-28 subido correctamente.
Batch 28-29 subido correctamente.
Batch 29-30 subido correctamente.

In [181]:
def chat_GPT(query):
    
    query_embedding = generate_embedding(client, query, embedding_model)
    # Construir consulta híbrida
    payload = {
      "search": query,
      "select": "title, subtitle, content, category", 
      "queryLanguage": "en-us",
      "vectorQueries": [
        {
          "kind": "vector",
          "vector": query_embedding,  
          "fields": "contentVector",
          "k": 3
        }
      ],
      "top": 10
    }
    
    # Enviar consulta a Azure AI Search
    url = f"{search_service_endpoint}/indexes/{index_name}/docs/search?api-version={api_search_version}"
    response = requests.post(url, headers=headers, json=payload)
    
    results = response.json()['value']
    context = " ".join([doc["category"] + ", " + doc["title"] + ", " + doc["subtitle"]  + ", " + doc["content"] for doc in results])
    
    # Generar respuesta con GPT-4o
    
    system_prompt = f"""Actua como una asistente virtual de la boda de Laura y Alberto y tu nombre es Sofia y resuelve 
                        cada una de las dudas del usuario con un tono amable y servicial. 
                        Utiliza como variable de contexto {context} y no respondas a preguntas diferentes a la variable de contexto. 
                        Cuando respondas con bullets coloca un emoji deacuerdo al contexto de cada uno.
                        Sugiere alguno de los temas mas relevantes del evento en cada una de tus respuestas para guiar al usuario.
                        Actua como un asistente capaz de guiar paso a paso al usuario desde su ubicacion actual a cada una de las 
                        ubicaciones que el usuario te solicite"""
    
    user_prompt = query
    gpt_response  = client.chat.completions.create(
        model = "gpt-4o",
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        max_tokens=5000,  
        temperature=0.7,  
        top_p=0.9,  
        frequency_penalty=0,  
        presence_penalty=0,
        stop=None,  
        stream=False  
    )
    
    return gpt_response.choices[0].message.content
    return results


In [211]:
query = "Hola puedes darme la ubicacion de la boda por favor"
response = chat_GPT(query)
print(response)

¡Hola! 😊 Estoy aquí para ayudarte con todo lo relacionado al transporte para la boda de Laura y Alberto. Aquí tienes los detalles más importantes:

### 🚍 **Servicio de transporte para la boda**
- **Punto de encuentro y hora de salida**:
  - **Lugar**: Walmart Juriquilla (junto a Plaza Uptown).
  - **Dirección**: P.º de la República 12402, Epigmenio González, 76147 Santiago de Querétaro, Qro.
  - [Ubicación del punto de encuentro](https://maps.app.goo.gl/SM3ZydN2Hi34rh7R8).
  - **Hora de partida**: 12:00 p.m. en punto.
  - **Sugerencia**: Llega a las 11:45 a.m. para abordar con calma.

- **Itinerario del transporte**:
  - 🚩 Salida de Walmart Juriquilla.
  - 🕊️ Traslado a la ceremonia religiosa / misa.
  - 🌳 Traslado a la recepción (jardín / terreno).
  - 🏨 Regreso al hotel al finalizar la fiesta.

- **¡Importante!** Llena el formulario para confirmar tu lugar en el transporte:
  - [Formulario del transporte](https://forms.gle/F7vtf8cbaZtijC4T9).

Si tienes dudas adicionales sobre cómo l